In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
name_df = pd.read_csv('name.basic.tsv', sep='\t')
name_df = name_df[['nconst','primaryName','knownForTitles']]

director_writers = pd.read_csv('title.crew.tsv', sep='\t')

In [2]:
name_df.head()

,nconst,primaryName,knownForTitles
0,nm0000001,Fred Astaire,"tt0031983,tt0072308,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,"tt0037382,tt0117057,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,"tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,"tt0072562,tt0080455,tt0078723,tt0077975"
4,nm0000005,Ingmar Bergman,"tt0050976,tt0060827,tt0083922,tt0050986"


In [144]:
director_writers.head()

,tconst,directors,writers
0,tt0000001,[nm0005690],[\N]
1,tt0000002,[nm0721526],[\N]
2,tt0000003,[nm0721526],[\N]
3,tt0000004,[nm0721526],[\N]
4,tt0000005,[nm0005690],[\N]


In [146]:
print(director_writers.info())
print(name_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8407399 entries, 0 to 8407398
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   tconst     object
 1   directors  object
 2   writers    object
dtypes: object(3)
memory usage: 192.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11359935 entries, 0 to 11359934
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   nconst          object
 1   primaryName     object
 2   knownForTitles  object
dtypes: object(3)
memory usage: 260.0+ MB
None


In [ ]:
rating_df = pd.read_csv('title.rating.tsv', sep='\t')
movie_df = pd.read_csv('title.basic.tsv', sep='\t')

In [145]:
print(movie_df.info())
print(rating_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8300681 entries, 0 to 8300680
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 570.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1189408 entries, 0 to 1189407
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1189408 non-null  object 
 1   averageRating  1189408 non-null  float64
 2   numVotes       1189408 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 27.2+ MB
None


In [4]:
movie_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
#drop null value from move (primary title and original title)
movie_df_clean = movie_df.loc[(movie_df['primaryTitle'].notnull())&(movie_df['originalTitle'].notnull())]
movie_df_clean = movie_df_clean.loc[movie_df['genres'].notnull()]
movie_df_clean.isnull().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64

In [6]:
movie_df_clean['startYear'] = movie_df_clean['startYear'].replace('\\N', np.nan)
movie_df_clean['startYear'] = movie_df_clean['startYear'].astype('float64')
movie_df_clean['endYear'] = movie_df_clean['endYear'].replace('\\N', np.nan)
movie_df_clean['endYear'] = movie_df_clean['endYear'].astype('float64')
movie_df_clean['runtimeMinutes'] = movie_df_clean['runtimeMinutes'].replace('\\N', np.nan)
print(movie_df_clean['endYear'].unique()[:5])

[  nan 1947. 1945. 1955. 1949.]


In [7]:
def transform_to_list(x):
    if ',' in x: 
    #ubah menjadi list apabila ada data pada kolom genre
        return x.split(',')
    else: 
    #jika tidak ada data, ubah menjadi list kosong
        return []

movie_df_clean['genres'] = movie_df_clean['genres'].apply(lambda x: transform_to_list(x))

In [8]:
#join movie_df + rating_df
movie_rating_df = pd.merge(movie_df_clean, rating_df, on='tconst', how='inner')

movie_rating_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5,"[Animation, Short]",6.0,232
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4,"[Animation, Comedy, Romance]",6.5,1577
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12,"[Animation, Short]",6.1,151
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399


In [9]:
print(name_df.head())
print(name_df.info())

      nconst      primaryName                           knownForTitles
0  nm0000001     Fred Astaire  tt0031983,tt0072308,tt0050419,tt0053137
1  nm0000002    Lauren Bacall  tt0037382,tt0117057,tt0038355,tt0071877
2  nm0000003  Brigitte Bardot  tt0057345,tt0056404,tt0054452,tt0049189
3  nm0000004     John Belushi  tt0072562,tt0080455,tt0078723,tt0077975
4  nm0000005   Ingmar Bergman  tt0050976,tt0060827,tt0083922,tt0050986
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11359935 entries, 0 to 11359934
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   nconst          object
 1   primaryName     object
 2   knownForTitles  object
dtypes: object(3)
memory usage: 260.0+ MB
None


In [10]:
print(director_writers.head())
print(director_writers.info())

      tconst  directors writers
0  tt0000001  nm0005690      \N
1  tt0000002  nm0721526      \N
2  tt0000003  nm0721526      \N
3  tt0000004  nm0721526      \N
4  tt0000005  nm0005690      \N
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8407399 entries, 0 to 8407398
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   tconst     object
 1   directors  object
 2   writers    object
dtypes: object(3)
memory usage: 192.4+ MB
None


In [11]:
#Convert to list
director_writers['directors'] = director_writers['directors'].apply(lambda row: row.split(','))
director_writers['writers'] = director_writers['writers'].apply(lambda row: row.split(','))

director_writers.head()

,tconst,directors,writers
0,tt0000001,[nm0005690],[\N]
1,tt0000002,[nm0721526],[\N]
2,tt0000003,[nm0721526],[\N]
3,tt0000004,[nm0721526],[\N]
4,tt0000005,[nm0005690],[\N]


In [12]:
#checking variance
print(name_df['knownForTitles'].apply(lambda x: len(x.split(','))).unique())

#convert to list 'knownForTitles'
name_df['knownForTitles'] = name_df['knownForTitles'].apply(lambda x: x.split(','))

print(name_df.head())

[4 1 3 2 5 6]
      nconst      primaryName                                knownForTitles
0  nm0000001     Fred Astaire  [tt0031983, tt0072308, tt0050419, tt0053137]
1  nm0000002    Lauren Bacall  [tt0037382, tt0117057, tt0038355, tt0071877]
2  nm0000003  Brigitte Bardot  [tt0057345, tt0056404, tt0054452, tt0049189]
3  nm0000004     John Belushi  [tt0072562, tt0080455, tt0078723, tt0077975]
4  nm0000005   Ingmar Bergman  [tt0050976, tt0060827, tt0083922, tt0050986]


In [13]:
df_uni = []

for x in ['knownForTitles']:
    #mset index
    idx = name_df.index.repeat(name_df['knownForTitles'].str.len())
   
   #break the list than combine it
    df1 = pd.DataFrame({
        x: np.concatenate(name_df[x].values)
    })
    
    #replace the index
    df1.index = idx
    #append the dataframe to list
    df_uni.append(df1)
    
#combine df
df_concat = pd.concat(df_uni, axis=1)

#left join with the original dataset
unnested_df = df_concat.join(name_df.drop(['knownForTitles'], 1), how='left')

unnested_df = unnested_df[name_df.columns.tolist()]
print(unnested_df)

             nconst         primaryName knownForTitles
0         nm0000001        Fred Astaire      tt0031983
0         nm0000001        Fred Astaire      tt0072308
0         nm0000001        Fred Astaire      tt0050419
0         nm0000001        Fred Astaire      tt0053137
1         nm0000002       Lauren Bacall      tt0037382
...             ...                 ...            ...
11359930  nm9993714   Romeo del Rosario     tt11657662
11359931  nm9993716       Essias Loberg             \N
11359932  nm9993717  Harikrishnan Rajan      tt8736744
11359933  nm9993718         Aayush Nair             \N
11359934  nm9993719          Andre Hill             \N

[19806599 rows x 3 columns]


In [14]:
unnested_drop = unnested_df.drop(['nconst'], axis=1)

#menyiapkan bucket untuk dataframe
df_uni = []

for col in ['primaryName']:
    #agregasi kolom PrimaryName sesuai group_col yang sudah di define di atas
    dfi = unnested_drop.groupby(['knownForTitles'])[col].apply(list)
    #Lakukan append
    df_uni.append(dfi)
df_grouped = pd.concat(df_uni, axis=1).reset_index()
df_grouped.columns = ['knownForTitles','cast_name']
print(df_grouped)

        knownForTitles                                          cast_name
0                   \N  [Eivind Fredagsvik Øyangen, James 52X, Elena A...
1            tt0000001                                       [Carmencita]
2            tt0000003  [Émile Reynaud, Gaston Paulin, Julien Pappé, T...
3            tt0000004                                    [Gaston Paulin]
4            tt0000005                         [Charles Kayser, John Ott]
...                ...                                                ...
1717505      tt9916734  [Juancho Rodriguez, Stephanie Gonzalez, Emelie...
1717506      tt9916754  [Delfina Rocha, José Albano, Odério Dias, Herm...
1717507      tt9916756                      [Brianna Laren, Tomeka Allen]
1717508      tt9916764      [Meleisha Edwards, Allegra Montgomery Walker]
1717509      tt9916856  [Kathrin Knöpfle, Ardian Hartono, Farahnaz Hat...

[1717510 rows x 2 columns]


In [15]:
#join between movie table and cast table 
base_df = pd.merge(df_grouped, movie_rating_df, left_on='knownForTitles', right_on='tconst', how='inner')

#join between base_df and director_writer table
base_df = pd.merge(base_df, director_writers, left_on='tconst', right_on='tconst', how='left')
base_df.head()

,knownForTitles,cast_name,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers
0,tt0000001,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N]
1,tt0000003,"[Émile Reynaud, Gaston Paulin, Julien Pappé, T...",tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892.0,NaN,4,"[Animation, Comedy, Romance]",6.5,1577,[nm0721526],[\N]
2,tt0000004,[Gaston Paulin],tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12,"[Animation, Short]",6.1,151,[nm0721526],[\N]
3,tt0000005,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N]
4,tt0000007,[Peter Courtney],tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894.0,NaN,1,"[Short, Sport]",5.4,750,"[nm0005690, nm0374658]",[\N]


In [16]:
#drop column knownForTitles (beacsue it's same with tconst)
base_drop = base_df.drop(['knownForTitles'], axis=1)
print(base_drop.info())

#change null values on genres to 'Unknown'
base_drop['genres'] = base_drop['genres'].fillna('Unknown')

print(base_drop.isnull().sum())

#change null value from director and writers to 'Unknown'
base_drop[['directors','writers']] = base_drop[['directors','writers']].fillna('unknown')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556583 entries, 0 to 556582
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   cast_name       556583 non-null  object 
 1   tconst          556583 non-null  object 
 2   titleType       556583 non-null  object 
 3   primaryTitle    556583 non-null  object 
 4   originalTitle   556583 non-null  object 
 5   isAdult         556583 non-null  object 
 6   startYear       556513 non-null  float64
 7   endYear         32305 non-null   float64
 8   runtimeMinutes  460732 non-null  object 
 9   genres          556583 non-null  object 
 10  averageRating   556583 non-null  float64
 11  numVotes        556583 non-null  int64  
 12  directors       556573 non-null  object 
 13  writers         556573 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 63.7+ MB
None
cast_name              0
tconst                 0
titleType              0
primaryTi

In [17]:
print(base_drop.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556583 entries, 0 to 556582
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   cast_name       556583 non-null  object 
 1   tconst          556583 non-null  object 
 2   titleType       556583 non-null  object 
 3   primaryTitle    556583 non-null  object 
 4   originalTitle   556583 non-null  object 
 5   isAdult         556583 non-null  object 
 6   startYear       556513 non-null  float64
 7   endYear         32305 non-null   float64
 8   runtimeMinutes  460732 non-null  object 
 9   genres          556583 non-null  object 
 10  averageRating   556583 non-null  float64
 11  numVotes        556583 non-null  int64  
 12  directors       556583 non-null  object 
 13  writers         556583 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 63.7+ MB
None


In [18]:
base_drop['combined'] = base_drop['directors']

In [19]:
columns = ['genres', 'cast_name', 'directors', 'writers']
l = []
for i in range(len(base_drop)):
    words = ''
    for col in columns:
        words += ' '.join(base_drop[col][i]) + ' '
    l.append(words)
l

['Documentary Short Carmencita nm0005690 \\N ',
 'Animation Comedy Romance Émile Reynaud Gaston Paulin Julien Pappé Tamara Pappé Claire Lissalde Sophie Nicolle nm0721526 \\N ',
 'Animation Short Gaston Paulin nm0721526 \\N ',
 'Comedy Short Charles Kayser John Ott nm0005690 \\N ',
 'Short Sport Peter Courtney nm0005690 nm0374658 \\N ',
 'Documentary Short Fred Ott nm0005690 \\N ',
 'Romance Short Blanche Bayliss Alexander Black William Courtenay Chauncey Depew nm0085156 nm0085156 ',
 'Documentary Short Grunato nm0804434 \\N ',
 'Documentary Short Mrs. Auguste Lumiere Auguste Lumière Madeleine Koehler Rose Lumière Marcel Koehler nm0525908 nm0525910 \\N ',
 'Documentary Short P.J.C. Janssen nm0525910 \\N ',
 'Comedy Short François Clerc Benoît Duval Louis Lumière nm0525910 \\N ',
 'Documentary Short Emil Skladanowsky Ploetz Larella nm1587194 nm0804434 \\N ',
 ' Delaware nm0804434 \\N ',
 'News Short Birt Acres Empress Augusta Victoria Kaiser Wilhelm II nm0010291 \\N ',
 'News Short Sport

In [20]:
base_drop['combined'] = l

In [21]:
#AKAS join
akas_df = pd.read_csv('title.akas.tsv', sep='\t')

C:\Users\drago\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
akas_df_reduced = akas_df[['titleId', 'region']]
akas_df_reduced.head()

,titleId,region
0,tt0000001,UA
1,tt0000001,DE
2,tt0000001,HU
3,tt0000001,GR
4,tt0000001,RU


In [23]:
new_base_drop = pd.merge(akas_df_reduced, base_drop, left_on='titleId', right_on='tconst', how='inner')
new_base_drop.head()

,titleId,region,cast_name,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,combined
0,tt0000001,UA,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N],Documentary Short Carmencita nm0005690 \N
1,tt0000001,DE,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N],Documentary Short Carmencita nm0005690 \N
2,tt0000001,HU,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N],Documentary Short Carmencita nm0005690 \N
3,tt0000001,GR,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N],Documentary Short Carmencita nm0005690 \N
4,tt0000001,RU,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N],Documentary Short Carmencita nm0005690 \N


In [24]:
US_base_drop = new_base_drop.loc[(new_base_drop['region']== 'US')]
US_base_drop.head()

,titleId,region,cast_name,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,combined
5,tt0000001,US,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N],Documentary Short Carmencita nm0005690 \N
27,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...
31,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...
32,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...
35,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...


In [25]:
US_base_drop.drop_duplicates(subset=['primaryTitle'])
US_base_drop.head()

,titleId,region,cast_name,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,combined
5,tt0000001,US,[Carmencita],tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1,"[Documentary, Short]",5.7,1822,[nm0005690],[\N],Documentary Short Carmencita nm0005690 \N
27,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...
31,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...
32,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...
35,tt0000005,US,"[Charles Kayser, John Ott]",tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1,"[Comedy, Short]",6.2,2399,[nm0005690],[\N],Comedy Short Charles Kayser John Ott nm0005690...


In [147]:
US_base_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341885 entries, 5 to 2775643
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   titleId         341885 non-null  object 
 1   region          341885 non-null  object 
 2   cast_name       341885 non-null  object 
 3   tconst          341885 non-null  object 
 4   titleType       341885 non-null  object 
 5   primaryTitle    341885 non-null  object 
 6   originalTitle   341885 non-null  object 
 7   isAdult         341885 non-null  object 
 8   startYear       341856 non-null  float64
 9   endYear         18331 non-null   float64
 10  runtimeMinutes  294972 non-null  object 
 11  genres          341885 non-null  object 
 12  averageRating   341885 non-null  float64
 13  numVotes        341885 non-null  int64  
 14  directors       341885 non-null  object 
 15  writers         341885 non-null  object 
 16  combined        341885 non-null  object 
 17  score    

In [26]:
def imdb_weighted_rating(US_base_drop, var=0.8):
    v = US_base_drop['numVotes']
    R = US_base_drop['averageRating']
    C = US_base_drop['averageRating'].mean()
    m = US_base_drop['numVotes'].quantile(var)
    US_base_drop['score'] = (v/(m+v))*R + (m/(m+v))*C
    return US_base_drop['score']
    
imdb_weighted_rating(US_base_drop)

<ipython-input-26-c17e435db445>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  US_base_drop['score'] = (v/(m+v))*R + (m/(m+v))*C


5          5.911426
27         6.255876
31         6.255876
32         6.255876
35         6.255876
             ...   
2775606    6.488232
2775615    6.406239
2775633    6.389976
2775641    6.261664
2775643    6.261664
Name: score, Length: 341885, dtype: float64

In [148]:
c = US_base_drop['averageRating'].mean()
m = US_base_drop['numVotes'].quantile(0.94)
print(m)
q_movies = US_base_drop.copy().loc[US_base_drop['numVotes'] >= m]

q_movies.info()

7438.879999999888
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20514 entries, 72 to 2775056
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   titleId         20514 non-null  object 
 1   region          20514 non-null  object 
 2   cast_name       20514 non-null  object 
 3   tconst          20514 non-null  object 
 4   titleType       20514 non-null  object 
 5   primaryTitle    20514 non-null  object 
 6   originalTitle   20514 non-null  object 
 7   isAdult         20514 non-null  object 
 8   startYear       20510 non-null  float64
 9   endYear         2169 non-null   float64
 10  runtimeMinutes  20331 non-null  object 
 11  genres          20514 non-null  object 
 12  averageRating   20514 non-null  float64
 13  numVotes        20514 non-null  int64  
 14  directors       20514 non-null  object 
 15  writers         20514 non-null  object 
 16  combined        20514 non-null  object 
 17  score     

In [111]:
q_movies.head()

,titleId,region,cast_name,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers,combined,score
72,tt0000012,US,"[Mrs. Auguste Lumiere, Auguste Lumière, Madele...",tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896.0,NaN,1,"[Documentary, Short]",7.4,11238,"[nm0525908, nm0525910]",[\N],Documentary Short Mrs. Auguste Lumiere Auguste...,7.341486
82,tt0000012,US,"[Mrs. Auguste Lumiere, Auguste Lumière, Madele...",tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896.0,NaN,1,"[Documentary, Short]",7.4,11238,"[nm0525908, nm0525910]",[\N],Documentary Short Mrs. Auguste Lumiere Auguste...,7.341486
436,tt0000417,US,"[Victor André, Henri Delannoy, Théophile Micha...",tt0000417,short,A Trip to the Moon,Le voyage dans la lune,0,1902.0,NaN,13,"[Action, Adventure, Comedy]",8.2,46409,[nm0617588],"[nm0617588, nm0894523, nm0920229]",Action Adventure Comedy Victor André Henri Del...,8.172769
454,tt0000417,US,"[Victor André, Henri Delannoy, Théophile Micha...",tt0000417,short,A Trip to the Moon,Le voyage dans la lune,0,1902.0,NaN,13,"[Action, Adventure, Comedy]",8.2,46409,[nm0617588],"[nm0617588, nm0894523, nm0920229]",Action Adventure Comedy Victor André Henri Del...,8.172769
457,tt0000417,US,"[Victor André, Henri Delannoy, Théophile Micha...",tt0000417,short,A Trip to the Moon,Le voyage dans la lune,0,1902.0,NaN,13,"[Action, Adventure, Comedy]",8.2,46409,[nm0617588],"[nm0617588, nm0894523, nm0920229]",Action Adventure Comedy Victor André Henri Del...,8.172769


In [112]:

df = q_movies[['primaryTitle', 'combined']]
df.head()

,primaryTitle,combined
72,The Arrival of a Train,Documentary Short Mrs. Auguste Lumiere Auguste...
82,The Arrival of a Train,Documentary Short Mrs. Auguste Lumiere Auguste...
436,A Trip to the Moon,Action Adventure Comedy Victor André Henri Del...
454,A Trip to the Moon,Action Adventure Comedy Victor André Henri Del...
457,A Trip to the Moon,Action Adventure Comedy Victor André Henri Del...


In [113]:
df = df.drop_duplicates()

In [114]:
#num = np.arange(0,10258)
df = df.reset_index()

In [115]:
df = df.drop(labels='index', axis=1)

In [116]:
df

,primaryTitle,combined
0,The Arrival of a Train,Documentary Short Mrs. Auguste Lumiere Auguste...
1,A Trip to the Moon,Action Adventure Comedy Victor André Henri Del...
2,The Great Train Robbery,Action Adventure Crime Walter Cameron Frank Ha...
3,The Birth of a Nation,Drama History War D.W. Griffith Spottiswoode A...
4,Intolerance,Drama History D.W. Griffith G.W. Bitzer Frank ...
...,...,...
13562,The Disappearance of Madeleine McCann,Crime Documentary Mystery Anthony Summers Kiyo...
13563,I Care a Lot,Comedy Crime Thriller Michele Baker Matthew Vo...
13564,Coffee & Kareem,Action Comedy Crime Rob Farrell Gordon Hashimo...
13565,Motherland: Fort Salem,Drama Fantasy Sci-Fi Arlen Aguayo-Stewart Gena...


In [117]:
features = df['combined'].str.split(' ')
features = features.fillna("").astype('str')
features



0        ['Documentary', 'Short', 'Mrs.', 'Auguste', 'L...
1        ['Action', 'Adventure', 'Comedy', 'Victor', 'A...
2        ['Action', 'Adventure', 'Crime', 'Walter', 'Ca...
3        ['Drama', 'History', 'War', 'D.W.', 'Griffith'...
4        ['Drama', 'History', 'D.W.', 'Griffith', 'G.W....
                               ...                        
13562    ['Crime', 'Documentary', 'Mystery', 'Anthony',...
13563    ['Comedy', 'Crime', 'Thriller', 'Michele', 'Ba...
13564    ['Action', 'Comedy', 'Crime', 'Rob', 'Farrell'...
13565    ['Drama', 'Fantasy', 'Sci-Fi', 'Arlen', 'Aguay...
13566    ['Action', 'Crime', 'Thriller', 'Chethan', 'Ar...
Name: combined, Length: 13567, dtype: object

In [118]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer (analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(features)
tfidf_matrix.shape

(13567, 3198391)

In [120]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.        , 0.        , 0.01668324],
       [0.        , 1.        , 0.00765878, 0.        ],
       [0.        , 0.00765878, 1.        , 0.00418124],
       [0.01668324, 0.        , 0.00418124, 1.        ]])

In [121]:
titles = df['primaryTitle']
indices = pd.Series(df.index, index = titles)

In [122]:
titles.index

RangeIndex(start=0, stop=13567, step=1)

In [123]:
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [124]:
titles

0                       The Arrival of a Train
1                           A Trip to the Moon
2                      The Great Train Robbery
3                        The Birth of a Nation
4                                  Intolerance
                         ...                  
13562    The Disappearance of Madeleine McCann
13563                             I Care a Lot
13564                          Coffee & Kareem
13565                   Motherland: Fort Salem
13566                                   Kaithi
Name: primaryTitle, Length: 13567, dtype: object

In [159]:
genre_recommendations("Friends").head(10)

5821                                   Joey
2530                            Law & Order
4554      Law & Order: Special Victims Unit
3492                         The Daily Show
1912                             EastEnders
4731                               Survivor
6588     Captain America: The First Avenger
11690                     Avengers: Endgame
692                              Doctor Who
5535                               I, Robot
Name: primaryTitle, dtype: object

In [142]:
genre_recommendations('Insidious').head(20)

10310                  Insidious: Chapter 2
2530                            Law & Order
692                              Doctor Who
4554      Law & Order: Special Victims Unit
6588     Captain America: The First Avenger
11690                     Avengers: Endgame
3492                         The Daily Show
1912                             EastEnders
6765                                Ant-Man
4731                               Survivor
10919                             Furious 7
9510                                The Bay
10562                            Dark Skies
6916                                 Avatar
10362                          Chicago Fire
10032               Guardians of the Galaxy
7236                           The Avengers
6058                            John Carter
8899                             Shark Tank
7414                        Game of Thrones
Name: primaryTitle, dtype: object